<center><img src='./Figs/cs-logo.png' width=200></center>



<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Lab 4 : Question Answering with BERT</center>
<hr style=" border:none; height:3px;">
</h1>

### The SQuAD v1.1 Benchmark

This task is the application of BERT to the Stanford Question Answering Dataset (SQuAD). The task posed by the SQuAD benchmark is a little different than you might think. Given a question, and **a passage of text containing the answer**, BERT needs to highlight the "span" of text corresponding to the correct answer. 

The SQuAD homepage has a fantastic tool for exploring the questions and reference text for this dataset, and even shows the predictions made by top-performing models. For example, here are some [interesting examples](https://rajpurkar.github.io/SQuAD-explorer/explore/1.1/dev/Super_Bowl_50.html?model=r-net+%20(ensemble)%20(Microsoft%20Research%20Asia)&version=1.1).

In this last lab, you will download a model that is *already fine-tuned* for question answering, and try it out on your own text. If you want to fine-tune on your own dataset, it is possible to fine-tune BERT for question answering yourself. See [run_squad.py](https://huggingface.co/transformers/v2.5.1/examples.html#squad) example. For example, you can try to fine-tune it an [FQuad](https://fquad.illuin.tech/) a question answering dataset in French.

**Note:** The example code in this lab is a commented and expanded version of the short example provided in the `transformers` documentation [here](https://huggingface.co/transformers/model_doc/bert.html?highlight=bertforquestionanswering#transformers.BertForQuestionAnswering).

### Input Format

For a Question Answering task, we need to pack both the question and the reference that contains the answer into a single continous input, where the two are seperated by a `[SEP]` token.

<img src="https://i.imgur.com/qtwZvtb.png" width="800">

The two pieces of text are separated by the special `[SEP]` token. The sentence embeddings (or segment embeddings) are also used to differentiate the question from the reference text. These are simply two embeddings (for sentences "A" and "B") that BERT learned, and which are added to the token embeddings before feeding them into the input layer.

### Start & End Token Classifiers

In order to detect the span of the reference test that contains the answer, BERT simply needs to predict which token marks the start of the answer, and which token marks the end.

To this end, you will have two classifiers, a start classifier that will give the probability the a given token is the start of the anwer, which is a simple dot product with a vector of the same size as the output embeddings followed by a softmax to get a probability distribution (we compare all of the output embeddings with the embeddings of a `[START]` embedding). The same is done for the `[END]`. The results are the position with the maximum probability for start and end.

<img src="https://i.imgur.com/aLf7jLu.png" width="800">

### Setup

As always, let's set the GPU and the get the transformer library.

In [ ]:
!pip install transformers

In [ ]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    assert False, "Please select GPU in the Colab"

### Load Fine-Tuned BERT-large

For Question Answering you will use the `BertForQuestionAnswering` class from the `transformers` library.
This class supports fine-tuning, but for this example we will keep things simpler and load a BERT model that has already been fine-tuned for the SQuAD benchmark.

The `transformers` library has a large collection of pre-trained models. The full list is in their documentation [here](https://huggingface.co/transformers/pretrained_models.html).

For Question Answering, they have a version of BERT-large that has already been fine-tuned for the SQuAD benchmark. BERT-large is really big... it has 24-layers and an embedding size of 1,024, for a total of 340M parameters. Instead of the base version, with 12 layers and embedding of 768. Totaling 1.34GB, so it takes a couple minutes to download. 

In [ ]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Load the tokenizer as well. The vocabulary of BERT large is identically to the one in bert-base-uncased. So we can load the tokenizer from `bert-base-uncased` and it works just as well.

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

### Asking a Question

Now you are ready to feed in an example! A QA example consists of a question and a passage of text containing the answer to that question.

Let's try an example:

In [ ]:
question = "What year did the first manned Apollo flight occur?"
answer_text = "The Apollo program, also known as Project Apollo, was the third United States human spaceflight program carried out by the National Aeronautics and Space Administration (NASA), which accomplished landing the first humans on the Moon from 1969 to 1972. First conceived during Dwight D. Eisenhower's administration as a three-man spacecraft to follow the one-man Project Mercury which put the first Americans in space, Apollo was later dedicated to President John F. Kennedy's national goal of "landing a man on the Moon and returning him safely to the Earth" by the end of the 1960s, which he proposed in a May 25, 1961, address to Congress. Project Mercury was followed by the two-man Project Gemini (1962–66). The first manned flight of Apollo was in 1968.."

##  <span style="color:red">Your turn. </span>

1. **Tokenize both the question and the answer text**
1. **Concatenate them, and place the special token `[SEP]` in between**
1. **Create segment IDs; a simple list, with 0s for the first sentence & `[SEP]`, which is the question in our case, and 1s for the rest. Must be of the same length as the input.**
1. **Conver the inputs to pytorch tensors.**
1. **Pass them to the model, the segment ids are passed as argument named `token_type_ids`.**
1. **With two outputs `start_scores, end_scores = model()`. Get the starting and ending positions.**
1. **Using the start and end positions, convert the ids tokens to tokens using `tokenizer.convert_ids_to_tokens`, then print the answer with the correct formatting.**
1. **Plot the output distributions for START and END positions.**

In [1]:
# Your turn